In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import fredapi as fa
fred = fa.Fred(api_key='b6bbb1bfe76d93bf0ca60a69b99d3474')

In [8]:
# bcdc_07192021.xlsx
# not sure what it is
pd.read_excel("../Data/bcdc_07192021.xlsx", skiprows=2).head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Peak month number,Trough month number,"Duration, peak to trough","Duration, trough to peak","Duration, trough to trough","Duration, peak to peak"
0,NaN,NaN,NaN,December 1854 (1854Q4),NaN,660.0,NaN,NaN,NaN,NaN
1,NaN,NaN,June 1857 (1857Q2),December 1858 (1858Q4),690.0,708.0,18.0,30.0,48.0,NaN
2,NaN,NaN,October 1860 (1860Q3),June 1861 (1861Q3),730.0,738.0,8.0,22.0,30.0,40.0
3,NaN,NaN,April 1865 (1865Q1),December 1867 (1868Q1),784.0,816.0,32.0,46.0,78.0,54.0
4,NaN,NaN,June 1869 (1869Q2),December 1870 (1870Q4),834.0,852.0,18.0,18.0,36.0,50.0


In [16]:
# GSCFI.xlsx
# GS Financial condition indicator
df = pd.read_excel("../Data/GSCFI.xlsx")
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", drop=True, inplace=True)
fig = px.line(df, title="Goldman Sachs Financial Conditions Indicator")
fig.show()
df.head()

,GSUSFCI Index
Date,
2022-09-30,100.81
2022-08-31,99.76
2022-07-29,99.23
2022-06-30,99.70
2022-05-31,98.89


## Walt's Code to get CPI and GDP

In [29]:
#pull data from FRED
fred = fa.Fred(api_key='b6bbb1bfe76d93bf0ca60a69b99d3474')

#CPI
# Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
cpi = fred.get_series('CPIAUCSL')
cpi.name = 'CPI'

# Make df for cpi
cpi_data = pd.DataFrame(index=cpi.index, data=cpi.values, columns=['CPI']).dropna()
# 
cpi_data['CPI YOY'] = cpi_data['CPI'].rolling(12).apply(lambda x: (x.iloc[-1] / x.iloc[0] - 1) * 100)
cpi_data['CPI'] = cpi_data['CPI YOY'].rolling(12).apply(lambda x: x.iloc[-1] - x.iloc[0] - 1)
cpi_data = cpi_data.dropna()

#GDP
gdp = fred.get_series('GDP')
gdp.name = 'GDP'

gdp_data = pd.DataFrame(index=gdp.index,data=gdp.values,columns=['GDP']).dropna()
gdp_data['GDP YOY'] = gdp_data['GDP'].rolling(4).apply(lambda x: (x.iloc[-1]/x.iloc[0] - 1) * 100)
gdp_data['GDP'] = gdp_data['GDP YOY'].rolling(4).apply(lambda x: x.iloc[-1] -x.iloc[0] - 1)
gdp_data = gdp_data.dropna()

#merge data
df = pd.concat([gdp_data, cpi_data], axis=1).dropna()

#make cycle
conditions = [
    (df['CPI'] < 0.0) & (df['GDP'] >= 0.0),
    (df['CPI'] >= 0.0) & (df['GDP'] >= 0.0), #& (df['GDP YOY'] > 0.5),
    (df['CPI'] < 0.0) & (df['GDP'] < 0.0)]
choices = [1.0, 2.0, 4.0]
df['cycle_gdp_cpi'] = np.select(conditions, choices, default=3.0)

#shift to get the 1 qtr
df = df.shift(1).dropna()
df

,GDP,GDP YOY,CPI,CPI YOY,cycle_gdp_cpi
1949-04-01,-9.302987,0.905099,-7.154492,1.436417,4.0
1949-07-01,-11.298352,-2.809854,-10.115786,-0.374844,4.0
1949-10-01,-9.169953,-2.666871,-10.949600,-2.988129,4.0
1950-01-01,-3.507446,-1.602347,-6.231905,-2.028146,4.0
1950-04-01,5.302378,3.492524,-4.417621,-1.672940,1.0
...,...,...,...,...,...
2021-04-01,21.434643,13.633221,-1.628993,1.232785,1.0
2021-07-01,8.470623,7.885368,2.984727,4.213031,2.0
2021-10-01,6.729891,8.503750,2.692618,4.855536,2.0
2022-01-01,-5.512110,9.121111,4.134840,6.086583,3.0


In [22]:
cpi = fred.get_series('CPIAUCSL')
cpi.name = 'CPI'
cpi

1947-01-01     21.480
1947-02-01     21.620
1947-03-01     22.000
1947-04-01     22.000
1947-05-01     21.950
               ...   
2022-04-01    288.663
2022-05-01    291.474
2022-06-01    295.328
2022-07-01    295.271
2022-08-01    295.620
Name: CPI, Length: 908, dtype: float64

In [23]:
cpi_data = pd.DataFrame(index=cpi.index,data=cpi.values,columns=['CPI']).dropna()
cpi_data['CPI YOY'] = cpi_data['CPI'].rolling(12).apply(lambda x: (x.iloc[-1]/x.iloc[0] - 1) *100 )
cpi_data['CPI'] = cpi_data['CPI YOY'].rolling(12).apply(lambda x: x.iloc[-1] -x.iloc[0] - 1)
cpi_data = cpi_data.dropna()
cpi_data

,CPI,CPI YOY
1948-11-01,-6.781343,3.203759
1948-12-01,-8.965715,1.562500
1949-01-01,-7.154492,1.436417
1949-02-01,-6.073501,1.744681
1949-03-01,-9.141528,0.377834
...,...,...
2022-04-01,2.044814,7.469871
2022-05-01,1.772095,7.572844
2022-06-01,2.647536,8.503071
2022-07-01,2.162755,8.121439


In [24]:
gdp = fred.get_series('GDP')
gdp.name = 'GDP'
gdp

1946-01-01          NaN
1946-04-01          NaN
1946-07-01          NaN
1946-10-01          NaN
1947-01-01      243.164
                ...    
2021-04-01    23046.934
2021-07-01    23550.420
2021-10-01    24349.121
2022-01-01    24740.480
2022-04-01    25248.476
Name: GDP, Length: 306, dtype: float64

In [25]:
gdp_data = pd.DataFrame(index=gdp.index,data=gdp.values,columns=['GDP']).dropna()
gdp_data['GDP YOY'] = gdp_data['GDP'].rolling(4).apply(lambda x: (x.iloc[-1]/x.iloc[0] - 1) *100 )
gdp_data['GDP'] = gdp_data['GDP YOY'].rolling(4).apply(lambda x: x.iloc[-1] -x.iloc[0] - 1)
gdp_data = gdp_data.dropna()
gdp_data

,GDP,GDP YOY
1948-07-01,-0.330356,7.488498
1948-10-01,-3.536175,5.503082
1949-01-01,-9.302987,0.905099
1949-04-01,-11.298352,-2.809854
1949-07-01,-9.169953,-2.666871
...,...,...
2021-04-01,8.470623,7.885368
2021-07-01,6.729891,8.503750
2021-10-01,-5.512110,9.121111
2022-01-01,-1.537119,7.348249


In [27]:
df = pd.concat([gdp_data, cpi_data], axis=1).dropna()
df

,GDP,GDP YOY,CPI,CPI YOY
1949-01-01,-9.302987,0.905099,-7.154492,1.436417
1949-04-01,-11.298352,-2.809854,-10.115786,-0.374844
1949-07-01,-9.169953,-2.666871,-10.949600,-2.988129
1949-10-01,-3.507446,-1.602347,-6.231905,-2.028146
1950-01-01,5.302378,3.492524,-4.417621,-1.672940
...,...,...,...,...
2021-04-01,8.470623,7.885368,2.984727,4.213031
2021-07-01,6.729891,8.503750,2.692618,4.855536
2021-10-01,-5.512110,9.121111,4.134840,6.086583
2022-01-01,-1.537119,7.348249,4.051159,7.058015


In [28]:
conditions = [
    (df['CPI'] < 0.0) & (df['GDP'] >= 0.0),
    (df['CPI'] >= 0.0) & (df['GDP'] >= 0.0), #& (df['GDP YOY'] > 0.5),
    (df['CPI'] < 0.0) & (df['GDP'] < 0.0)]
choices = [1.0, 2.0, 4.0]
df['cycle_gdp_cpi'] = np.select(conditions, choices, default=3.0)

#shift to get the 1 qtr
df = df.shift(1).dropna()
df

,GDP,GDP YOY,CPI,CPI YOY,cycle_gdp_cpi
1949-04-01,-9.302987,0.905099,-7.154492,1.436417,4.0
1949-07-01,-11.298352,-2.809854,-10.115786,-0.374844,4.0
1949-10-01,-9.169953,-2.666871,-10.949600,-2.988129,4.0
1950-01-01,-3.507446,-1.602347,-6.231905,-2.028146,4.0
1950-04-01,5.302378,3.492524,-4.417621,-1.672940,1.0
...,...,...,...,...,...
2021-04-01,21.434643,13.633221,-1.628993,1.232785,1.0
2021-07-01,8.470623,7.885368,2.984727,4.213031,2.0
2021-10-01,6.729891,8.503750,2.692618,4.855536,2.0
2022-01-01,-5.512110,9.121111,4.134840,6.086583,3.0
